In [1]:
pip install langchain openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 960.7/960.7 kB 10.1 MB/s eta 0:00:00a 0:00:01
  Using cached tenacity-8.2.2-py3-none-any.whl (24 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 23.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 7.2 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 8.0.1
    Uninstalling tenacity-8.0.1:
      Successfully uninstalled tenacity-8.0.1
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 20.0 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184490 sha256=71930fb3f306af90ff801b6d51b7afdabcda503a760c1ced1a3b1e0bc9d18d3f
  Stored in directory: /Users/ahmadbader/Library/Caches/pip/wheels/65/e1/9b/0c38fe6cfe02a9fe31cb6b4efd90985f17354d7f77872f2def
Successfully built python-docx
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.5 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import openai
import docx
from PyPDF2 import PdfReader

# Set up OpenAI API credentials
openai.api_key = 'YOUR_OPENAI_API_KEY'

def generate_default_answer(question):
    # List of common default answers
    default_answers = [
        "I'm sorry, but I don't have an answer for that.",
        "I'm afraid I couldn't find the information you're looking for.",
        "Apologies, I don't have the answer to that question.",
        "Unfortunately, I couldn't find any relevant information to answer your question.",
        "Regrettably, I don't have the necessary information to provide an answer.",
    ]
    
    # Choose a random default answer from the list
    default_answer = random.choice(default_answers)
    
    return default_answer


# Function to extract text from Word documents
def extract_text_from_docx(file_path):
    doc = docx.Document(file_path)
    text = ' '.join([paragraph.text for paragraph in doc.paragraphs])
    return text

# Function to extract text from PDF files
def extract_text_from_pdf(file_path):
    with open(file_path, 'rb') as file:
        reader = PdfReader(file)
        text = ''
        for page in reader.pages:
            text += page.extract_text()
    return text

# Function to process uploaded files and generate an answer
def generate_answer(files, question):
    content = ""
    for file_path in files:
        if file_path.endswith(".docx"):
            extracted_text = extract_text_from_docx(file_path)
        elif file_path.endswith(".pdf"):
            extracted_text = extract_text_from_pdf(file_path)
        else:
            print(f"Unsupported file format: {file_path}")
            continue
        
        content += extracted_text + "\n"

    # Prepare input for the model
    prompt = f"Question: {question}\nFiles Content: {content}\nAnswer:"
    
    # Generate answer using GPT-3.5 model
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=100,
        temperature=0.7,
        n=1,
        stop=None,
        timeout=30
    )
    
    # Extract the generated answer from the response
    answer = response.choices[0].text.strip()
    
    # If the answer is empty, generate a default answer
    if not answer:
        answer = generate_default_answer(question)
    
    return answer

# Example usage
question = input("Ask a question: ")
file_paths = input("Enter the paths of the files (separated by commas): ").split(",")

answer = generate_answer(file_paths, question)
print("Answer:", answer)
